# OSC communication
With the OSC communication module of sc3nb you can directly send and receive OSC packets.

Open Sound Control (OSC) is a networking protocol for sound and is used by SuperCollider to communicate between sclang and scsynth. sc3nb is itself a OSC client and server. This allows sc3nb to send and receive OSC traffic.

For more information on OSC and especially how Supercollider handles OSC packets please refer to the following links:
* [Open Sound Control Specification](http://opensoundcontrol.org/spec-1_0)
* [Server vs Client SuperCollider Guide](http://doc.sccode.org/Guides/ClientVsServer.html)
* [Server Command Reference](https://doc.sccode.org/Reference/Server-Command-Reference.html)
* [SuperCollider Synth Server Architecture](https://doc.sccode.org/Reference/Server-Architecture.html)

In [ ]:
import sc3nb as scn

To see more information what messages are send and received set the logging level to INFO as seen below.

In [ ]:
import logging
logging.basicConfig(level=logging.INFO)
# even more verbose logging is also avaible
# logging.basicConfig(level=logging.DEBUG)

In [ ]:
sc = scn.startup()

sc3nb serves as OSC server and as client of the SuperCollider server `scsynth`.<br>
You can also communicate with the SuperCollider interpreter `sclang`.<br>


You can see the current connection information with `sc.get_connection_info()`

In [ ]:
sc.get_connection_info()

## Types of messages
sc3nb is has 3 types of messages
* async messages
* message pairs
* all other messages


### async messages
* async messages are messages that are handled in an async fashion by scsynth.
* All async messages known to sc3nb are stored in `sc.osc.async_msgs`

In [ ]:
sc.osc.async_msgs

If msg is called with `sync=True` (default) sc3nb will perform a `sc.sync()` after the messages was send

In [ ]:
help(sc.sync)

Here is a example for the usage of `sc.msg` with an async message

In [ ]:
# /b_alloc has following arguments
# int    buffer number
# int    number of frames
# int    number of channels (optional. default = 1 channel)
# bytes  an OSC message to execute upon completion. (optional)
numFrames = 100
numChannels = 2
msg = scn.build_message("/s_new", ["s1", sc.next_node_id(), 1, 0, "freq", 300])

sc.msg("/b_alloc", [sc.next_buffer_id(), numFrames, numChannels, msg.dgram], sync=True)

### Message Pairs
* Message pairs are messages that have a reply address. 
* All message pairs known by sc3nb are stored in `sc.osc.msg_pairs`.
* These are the only messages that can be received. See below for more information on receiving messages and how to receive messages at custom OSC address 

In [ ]:
sc.osc.msg_pairs

## Usage

### Sending

To send OSC messages you can use sc.msg

In [ ]:
help(sc.msg)

You can also send OSC bundles.

In [ ]:
help(sc.bundle)

You can use `sc.bundle` to build a bundle with more messages or bundles

In [ ]:
bundle = sc.bundle(0)

bundle.add_msg("/s_new", ["s2", -1, 1, 0, "freq", 500])
bundle.add_msg("/n_run", [-1, 0])
bundle.send()

bundle = sc.bundle(1.0)
bundle.add_msg("/n_set", [-1, "freq", 200])
bundle.add_msg("/n_run", [-1, 1])
bundle.send()

bundle = sc.bundle(1.5, "/n_set", [-1, "freq", 400])
bundle.send()

bundle = sc.bundle(2.0)
bundle.add_msg("/n_free", [-1])
bundle.send()

**Attention:** If you care about precise timings you should provide a explict timetag (>1e6)

In [ ]:
import time
now = time.time()

bundle = sc.bundle(now)
bundle.add_msg("/s_new", ["s2", -1, 1, 0, "freq", 500])\
    .add_msg("/n_run", [-1, 0]).send()

bundle = sc.bundle(now + 1)
bundle.add_msg("/n_set", [-1, "freq", 200]).add_msg("/n_run", [-1, 1]).send()

bundle = sc.bundle(now + 1.5, "/n_set", [-1, "freq", 400]).send()

bundle = sc.bundle(now + 2).add_msg("/n_free", [-1]).send()

Even better: You can also add bundles to a bundle.

In [ ]:
now = time.time()
s2_node_id = sc.next_node_id()

bundle = sc.bundle(now)

inner_bundle = sc.bundle(now + 0.7)\
    .add_msg("/s_new", ["s1", -1, 1, 0, "freq", 400])\
    .add_msg("/n_query", [-1])\
    .add_msg("/s_new", ["s2", s2_node_id, 1, 0, "freq", 100])\
    .add_msg("/n_query", [-1])
bundle.add_content(inner_bundle.build())

ib2 = sc.bundle(now + 1.2)\
    .add_msg("/n_set", [-1, "freq", 200])\
    .add_msg("/s_new", ["s1", -1, 1, 0, "freq", 800])
bundle.add_content(ib2.build())

bundle.add_content(sc.bundle(now + 1.5).add_msg("/n_free", [s2_node_id]).build())

bundle.send()

Note that we only receive the n_go / n_end notifications from nodes with specified node ids

### Receiving

When messages are received they are stored in one of the message queues of the OSC communication module of sc3nb

You can see all queues and their OSC address with the help of `sc.msg_queues`. There is a queue for each message pair in `sc.msg_pairs`.

In [ ]:
sc.msg_queues

If we send a message to one of this addresses we receive the reply as return value

In [ ]:
sc.msg("/sync", 1500)

If we specify `sync=False` the message will be kept in the queue

In [ ]:
sc.msg("/sync", 42, sync=False)

In [ ]:
sc.msg_queues["/sync"]

You can see how many values were hold.

In [ ]:
sc.msg_queues["/sync"].skips

Notice that these hold messages can be skipped. 

In [ ]:
sc.sync()
sc.msg_queues["/sync"].skips

In [ ]:
sc.msg("/status", sync=False)

In [ ]:
sc.msg_queues["/status"]

In [ ]:
sc.msg("/status")

Therefore you should retrieve them with `get` if you care for old values in the queue and dont want them to be skipped.

In [ ]:
sc.msg("/status", sync=False)

In [ ]:
sc.msg_queues["/status"].get()

In [ ]:
sc.msg("/status")

To add a new message queue you can simply use `sc.update_msg_queues()`

In [ ]:
help(sc.update_msg_queues)

In [ ]:
sc.update_msg_queues({"/test": "/test.reply", "/time": "/time.reply"})

This updates the `msg_pairs`

In [ ]:
sc.osc.msg_pairs

You can now use the msg_queue of `/test`

In [ ]:
sc.msg_queues["/test"]

Lets use `OSCdef` in sclang to send us replies.

In [ ]:
%%sc
OSCdef.newMatching("test", {|msg, time, addr, recvPort| addr.sendMsg("/test.reply", "Hello there!")}, '/test');

In [ ]:
sc.msg("/test", sclang=True)

In [ ]:
%%sc
OSCdef.newMatching('mykey', {|msg, time, addr, recvPort| addr.sendMsg("/time.reply", time)}, '/time');

In [ ]:
sc.msg("/time", sclang=True, sync=False)
sc.msg("/time", sclang=True, sync=False)

In [ ]:
sc.msg_queues["/time"]

In [ ]:
print(sc.msg_queues["/time"].get())
print(sc.msg_queues["/time"].get())

In [ ]:
sc.msg_queues["/time"]

In [ ]:
%sc OSCdef('mykey').disable // to disable the OSCdef